This notebook will show you how to write three different kinds of mnist mnist classifiers:
* a linear model, which will achieve about 90% accuracy
* a fully connected two layer neural network (about 96% accuracy)
* a convolutional neural network (about 99% accuracy)

The goal is to keep the code to a minimum.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as plt

sess = None
def ResetSession():
    tf.reset_default_graph()
    global sess
    if sess is not None: sess.close()
    sess = tf.InteractiveSession()

Uncomment these cells to understand the format of the mnist dataset

In [ ]:
# How many examples do we have?
#learn = tf.contrib.learn
#mnist = learn.datasets.load_dataset('mnist')
#print ('Train, validation, test: %d, %d, %d' %  
#       (len(mnist.train.images), len(mnist.validation.images), len(mnist.test.images)))

# What's the format of a label?
# Notice these are "one-hot".
# This tells us this digit is a 7.

#print (mnist.train.labels[0])

# What's the format of an image? 
# This is an array of 28*28 = 784 floating point values.

#print (len(mnist.train.images[0]))

# To display an image, reshape it to 28x28

#plt.imshow(mnist.train.images[9].reshape((28,28)), cmap=plt.cm.gray_r)    

# Linear classifier

Code follows https://www.tensorflow.org/versions/r0.11/tutorials/index.html#mnist-for-ml-beginners

In [ ]:
ResetSession()

NUM_CLASSES = 10
NUM_PIXELS = 28 * 28
TRAIN_STEPS = 1000

mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

# Create the model
x = tf.placeholder(tf.float32, [None, NUM_PIXELS])
W = tf.Variable(tf.zeros([NUM_PIXELS, NUM_CLASSES]))
b = tf.Variable(tf.zeros([NUM_CLASSES]))
y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, NUM_CLASSES])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Train
tf.initialize_all_variables().run()
for i in range(TRAIN_STEPS):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                                  
print("Accuracy %f" % sess.run(accuracy, feed_dict={x: mnist.test.images,
                                  y_: mnist.test.labels}))

# Two layer fully connected deep neural network.

In [ ]:
ResetSession()

mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

NUM_CLASSES = 10
NUM_PIXELS = 28 * 28
TRAIN_STEPS = 1000
HIDDEN1_UNITS = 128
HIDDEN2_UNITS = 32

x = tf.placeholder(tf.float32, [None, NUM_PIXELS])

def weight_variable(inputs, outputs):
  # why initialize weights this way?
  # see http://cs231n.github.io/neural-networks-2/
    initial = tf.truncated_normal(shape=[inputs, outputs], stddev=1.0 / math.sqrt(float(inputs)))
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape=[shape])
    return tf.Variable(initial)

weights1 = weight_variable(NUM_PIXELS, HIDDEN1_UNITS)
biases1 = bias_variable(HIDDEN1_UNITS)
hidden1 = tf.nn.relu(tf.matmul(x, weights1) + biases1)

weights2 = weight_variable(HIDDEN1_UNITS, HIDDEN2_UNITS)
biases2 = bias_variable(HIDDEN2_UNITS)
hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)

weights3 = weight_variable(HIDDEN2_UNITS, NUM_CLASSES)
biases3 = bias_variable(NUM_CLASSES)

y = tf.matmul(hidden2, weights3) + biases3
y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess.run(tf.initialize_all_variables())

for i in range(TRAIN_STEPS):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    _, loss = sess.run([train_step, cross_entropy], feed_dict={x: batch_xs, y_: batch_ys})

    if i % 200 == 0: print ("loss %f" % loss)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Accuracy %f" % sess.run(accuracy, feed_dict={x: mnist.test.images,
                                  y_: mnist.test.labels}))

# Convolutional neural network

Code follows https://www.tensorflow.org/versions/r0.11/tutorials/mnist/pros/index.html#deep-mnist-for-experts

This will take some time to train.

In [ ]:
ResetSession()

mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# unlike the previous two classifiers, here we'll
# exploit the 2d structure of the image
x_image = tf.reshape(x, [-1,28,28,1])

# compute 32 features for each 5x5 patch on the image
# there's only 1 input channel at this point 
# if we had a R,G,B image, we'd have three
# at this point, the image size is 28x28
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# after this max_pool, the image size will be 14x14
h_pool1 = max_pool_2x2(h_conv1)

# now we'll compute 64 features for each 5x5 patch
# this time, our "patches" will look through the 
# entire depth of the image, so they're really 5x5x32
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# after this, we'll have a 7x7 image with 64 channels
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# we flatten that out and add a fully connected layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())

for i in range(20001):
    batch = mnist.train.next_batch(50)
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"% (i, train_accuracy)) 
    
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    if i%1000 == 0:
        print("test accuracy %g"%accuracy.eval(feed_dict={
            x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))